In [7]:
import os
os.environ['QT_API'] = 'PyQt6'

import pandas as pd
import numpy as np
import plotnine as pn
from janitor import clean_names
from matplotlib import rcParams
import seaborn as sns
import matplotlib.pyplot as plt
from pyhere import here
import pyarrow as pa
import pyarrow.parquet as pq

# Set some options
pd.set_option('display.max_columns', None)
pd.set_option('mode.copy_on_write', True)
rcParams.update({'savefig.bbox': 'tight'}) # Keeps plotnine legend from being cut off

In [9]:
cog1 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/CFQ_G.xpt')
cog2 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/CFQ_H.xpt')

hdl1 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/HDL_G.xpt')
hdl2 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/HDL_H.xpt')

ldl1 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/TRIGLY_G.xpt')
ldl2 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/TRIGLY_H.xpt')

bio1 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2011/DataFiles/BIOPRO_G.xpt')
bio2 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/BIOPRO_H.xpt')

In [12]:
bio_column_names = {
  'SEQN': 'id',
  'LBXSASSI': 'ast_u_l', # direct links to cardiovascular issues
  'LBXSCH': 'cholesterol_mg_dl',
  'LBXSCK': 'cpk_iu_l',
  'LBXSGL': 'glucose_mg_dl',
  'LBXSLDSI': 'ldh_u_l',
  'LBXSTR': 'tri_mg_dl',
  'LBXSBU': 'bun_g_dl', # indrect links to cardiovascular issues
  'LBXSCLSI': 'chloride_mmol_l',
  'LBXSKSI': 'potassium_mmol_l',
  'LBXSNASI': 'sodium_mmol_l',
  'LBXSUA': 'uric_acid_mg_dl',
  'LBXSC3SI': 'bicarbonate_mmol_l',
  'LBXSAL': 'albumin_g_dl' # indirect link to cognitive issues
  }

bio1 = bio1.rename(columns = bio_column_names)
bio2 = bio2.rename(columns = bio_column_names)

bio1 = bio1[['id',
             'ast_u_l',
             'cholesterol_mg_dl',
             'cpk_iu_l',
             'glucose_mg_dl',
             'ldh_u_l',
             'tri_mg_dl',
             'bun_g_dl',
             'chloride_mmol_l',
             'potassium_mmol_l',
             'sodium_mmol_l',
             'uric_acid_mg_dl',
             'bicarbonate_mmol_l',
             'albumin_g_dl']]

bio2 = bio2[['id', 
             'ast_u_l', 
             'cholesterol_mg_dl', 
             'cpk_iu_l', 
             'glucose_mg_dl', 
             'ldh_u_l', 
             'tri_mg_dl', 
             'bun_g_dl', 
             'chloride_mmol_l', 
             'potassium_mmol_l', 
             'sodium_mmol_l', 
             'uric_acid_mg_dl', 
             'bicarbonate_mmol_l', 
             'albumin_g_dl']]

bio = pd.concat([bio1, bio2])
bio.head()


,id,ast_u_l,cholesterol_mg_dl,cpk_iu_l,glucose_mg_dl,ldh_u_l,tri_mg_dl,bun_g_dl,chloride_mmol_l,potassium_mmol_l,sodium_mmol_l,uric_acid_mg_dl,bicarbonate_mmol_l,albumin_g_dl
0,62161.0,25.0,168.0,108.0,87.0,114.0,82.0,14.0,100.0,3.9,137.0,4.9,26.0,4.8
1,62163.0,19.0,158.0,147.0,64.0,137.0,114.0,16.0,104.0,3.8,137.0,5.8,26.0,4.0
2,62164.0,37.0,196.0,494.0,71.0,145.0,53.0,5.0,104.0,4.3,138.0,4.5,20.0,3.7
3,62165.0,19.0,165.0,135.0,87.0,145.0,68.0,12.0,105.0,4.0,139.0,5.8,26.0,4.1
4,62169.0,17.0,135.0,80.0,87.0,121.0,75.0,16.0,103.0,4.4,139.0,5.4,25.0,4.4


In [16]:
hdl_column_names = {'SEQN': 'id',
                    'LBDHDDSI': 'hdl_mg_dl'}

hdl1 = hdl1.rename(columns = hdl_column_names)
hdl2 = hdl2.rename(columns = hdl_column_names)

hdl1 = hdl1[['id', 'hdl_mg_dl']]
hdl2 = hdl2[['id', 'hdl_mg_dl']]

hdl = pd.concat([hdl1, hdl2])
hdl.head()

,id,hdl_mg_dl
0,62161.0,1.06
1,62163.0,1.14
2,62164.0,0.72
3,62165.0,1.63
4,62166.0,NaN


In [ ]:
ldl_column_names = {'SEQN': 'id',
                    'LBXTR': 'trigly_mg_dl',
                    'LBDLDL': 'ldl_mg_dl'}

ldl1 = ldl1.rename(columns = ldl_column_names)
ldl2 = ldl2.rename(columns = ldl_column_names)

ldl1 = ldl1[['id', 'trigly_mg_dl', 'ldl_mg_dl']]
ldl2 = ldl2[['id', 'trigly_mg_dl', 'ldl_mg_dl']]

ldl = pd.concat([ldl1, ldl2])

ldl.head()

,id,trigly_mg_dl,ldl_mg_dl
0,62161.0,84.0,110.0
1,62164.0,56.0,151.0
2,62165.0,71.0,84.0
3,62169.0,78.0,73.0
4,62170.0,44.0,77.0


In [25]:
cog_column_names = {'SEQN': 'id',
'CFDCCS': 'cerad_complete_status', # must be 1 to be involved
'CFDCST1': 'cerad_score_trial1_recall',
'CFDCST2': 'cerad_score_trial2_recall',
'CFDCST3': 'cerad_score_trial3_recall',
'CFDCSR': 'cerad_score_delay_recall',
'CFDCIT1': 'cerad_intrusion_wordcount_trial1',
'CFDCIT2': 'cerad_intrusion_wordcount_trial2',
'CFDCIT3': 'cerad_intrusion_wordcount_trial3',
'CFDCIR': 'cerad_intrusion_wordcount_recall',
'CFDAPP': 'animal_fluency_sample_test', # must be 1 to be involved
'CFDAST': 'animal_fluency_score',
'CFDDPP': 'digit_symbol_sample_test',
'CFDDS': 'digit_symbol_score'}

cog1 = cog1.rename(columns = cog_column_names)
cog2 = cog2.rename(columns = cog_column_names)

cog1 = cog1[['id',
             'cerad_complete_status',
             'cerad_score_trial1_recall',
             'cerad_score_trial2_recall',
             'cerad_score_trial3_recall',
             'cerad_score_delay_recall',
             'cerad_intrusion_wordcount_trial1',
             'cerad_intrusion_wordcount_trial2',
             'cerad_intrusion_wordcount_trial3',
             'cerad_intrusion_wordcount_recall',
             'animal_fluency_sample_test',
             'animal_fluency_score',
             'digit_symbol_sample_test',
             'digit_symbol_score']] 

cog2 = cog2[['id',
             'cerad_complete_status',
             'cerad_score_trial1_recall',
             'cerad_score_trial2_recall',
             'cerad_score_trial3_recall',
             'cerad_score_delay_recall',
             'cerad_intrusion_wordcount_trial1',
             'cerad_intrusion_wordcount_trial2',
             'cerad_intrusion_wordcount_trial3',
             'cerad_intrusion_wordcount_recall',
             'animal_fluency_sample_test',
             'animal_fluency_score',
             'digit_symbol_sample_test',
             'digit_symbol_score']] 

cog = pd.concat([cog1, cog2])

cog.head()

,id,cerad_complete_status,cerad_score_trial1_recall,cerad_score_trial2_recall,cerad_score_trial3_recall,cerad_score_delay_recall,cerad_intrusion_wordcount_trial1,cerad_intrusion_wordcount_trial2,cerad_intrusion_wordcount_trial3,cerad_intrusion_wordcount_recall,animal_fluency_sample_test,animal_fluency_score,digit_symbol_sample_test,digit_symbol_score
0,62174.0,1.0,3.0,6.0,7.0,5.0,5.397605e-79,5.397605e-79,5.397605e-79,1.000000e+00,1.0,18.0,1.0,33.0
1,62178.0,1.0,3.0,5.0,5.0,3.0,5.397605e-79,5.397605e-79,5.397605e-79,1.000000e+00,1.0,12.0,1.0,38.0
2,62191.0,1.0,4.0,5.0,7.0,5.0,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,1.0,17.0,1.0,26.0
3,62209.0,1.0,3.0,3.0,5.0,3.0,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,1.0,15.0,2.0,NaN
4,62215.0,1.0,5.0,8.0,8.0,7.0,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,1.0,15.0,1.0,47.0


In [26]:
df = (
  hdl
  .merge(ldl, 'left', 'id')
  .merge(bio, 'left', 'id')
  .merge(cog, 'right', 'id')
)


In [27]:
pct_miss = df.isnull().sum()/len(df)*100
pct_miss.sort_values(ascending = False)

ldl_mg_dl                           54.118664
trigly_mg_dl                        53.600230
digit_symbol_score                  13.191244
animal_fluency_score                10.426267
cerad_intrusion_wordcount_recall     9.965438
cerad_score_delay_recall             9.965438
cerad_score_trial3_recall            9.792627
cerad_intrusion_wordcount_trial3     9.792627
cerad_intrusion_wordcount_trial2     9.533410
cerad_score_trial2_recall            9.533410
cerad_intrusion_wordcount_trial1     9.331797
cerad_score_trial1_recall            9.331797
cerad_complete_status                7.949309
animal_fluency_sample_test           7.949309
digit_symbol_sample_test             7.949309
ast_u_l                              7.258065
tri_mg_dl                            7.229263
ldh_u_l                              7.229263
potassium_mmol_l                     7.200461
cholesterol_mg_dl                    7.171659
bun_g_dl                             7.171659
chloride_mmol_l                   

In [29]:
corr_matrix = df.corr()
corr_matrix = corr_matrix.round(2)

upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
threshold = 0.7
high_corr_pairs = upper_tri[upper_tri > threshold].stack().sort_values(ascending=False)
high_corr_pairs

trigly_mg_dl               tri_mg_dl                    0.98
ldl_mg_dl                  cholesterol_mg_dl            0.91
cerad_score_trial2_recall  cerad_score_trial3_recall    0.74
dtype: float64

In [ ]:
df = df.drop(columns = ['trigly_mg_dl', 'cholesterol_mg_dl'])
df.head()

In [31]:
df = df.round(2)

In [ ]:
df.head()

In [38]:
df = df.loc[(df['cerad_complete_status'] == 1) &
       (df['animal_fluency_sample_test'] == 1) &
       (df['digit_symbol_sample_test']) == 1]

In [ ]:
# df.to_csv(here('data/nhanes_2011_2013_cardio_cog.csv'))

# Start Here to Use Imputed Datasets (50 Datasets w/30 iterations each)

In [ ]:
imp_df = pd.read_csv(here('data/nhanes_2011_2013_cardio_cog_imputations_long.csv'))

In [9]:
imp_df = imp_df.drop(columns = ['cerad_complete_status', 'animal_fluency_sample_test', 'digit_symbol_sample_test'])

In [11]:
df_list = []
for i in imp_df['.imp'].unique():
  df = imp_df.loc[imp_df['.imp'] == i].drop(columns = ['id', '.imp', '.id'])
  
  df_list.append(df)

In [ ]:
df_list[0].describe().transpose()[['mean', 'min', 'max']]

In [20]:
df_list[0][['cerad_score_trial1_recall',
'cerad_score_trial2_recall',
'cerad_score_trial3_recall',
'cerad_score_delay_recall',
'cerad_intrusion_wordcount_trial1',
'cerad_intrusion_wordcount_trial2',
'cerad_intrusion_wordcount_trial3',
'cerad_intrusion_wordcount_recall']] = df_list[0][['cerad_score_trial1_recall',
'cerad_score_trial2_recall',
'cerad_score_trial3_recall',
'cerad_score_delay_recall',
'cerad_intrusion_wordcount_trial1',
'cerad_intrusion_wordcount_trial2',
'cerad_intrusion_wordcount_trial3',
'cerad_intrusion_wordcount_recall']].astype('category')

In [ ]:
df_list[0].info()

In [ ]:
cat_columns = df_list[0].select_dtypes('category')
num_columns = df_list[0].select_dtypes(['float', 'int'])

print(cat_columns.head(), '\n')
print(num_columns.head())